In [1]:
import numpy as np
import string
import time
import os

import matplotlib
matplotlib.use('Agg')
from matplotlib import pyplot as plt

import emotion_model
import dwdii_transforms

Using Theano backend.


In [2]:
imagePath = "C:\Code\Other\facial_expressions\images"
dataPath = "/root/facial_expressions/data/legend.csv"

In [3]:
emoMetaData = emotion_model.load_training_metadata(dataPath)

In [4]:
len(emoMetaData)

13683

In [5]:
emoMetaData.keys()[:10]

['King_Abdullah_II_0001.jpg',
 'Manuel_Poggiali_0002.jpg',
 'Paula_Radcliffe_0006.jpg',
 'Jean_Chretien_0048.jpg',
 'Scott_Rolen_0001.jpg',
 'Kate_Capshaw_0001.jpg',
 'David_Howard_0001.jpg',
 'Sheila_Wellstone_0001.jpg',
 'Yannos_Papantoniou_0001.jpg',
 'Kelly_Clarkson_0003.jpg']

In [ ]:
model = emotion_model.emotion_model_v1(True)

In [ ]:
os.listdir('/root/facial_expressions/data')

### References

* OpenCV/CV2: http://askubuntu.com/questions/447409/how-to-install-opencv-2-9-for-python
* Docker Commit: http://stackoverflow.com/questions/19585028/i-lose-my-data-when-the-container-exits